In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import pyautogui as py
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options

def convert_currency(currency_string):
    # 통화 기호와 쉼표 제거
    number_string = currency_string.replace('₩', '').replace(',', '')
    # 숫자로 변환하여 반환
    return int(number_string)


options = Options()
# options.add_argument('--start-maximized')
# options.add_experimental_option('detach', True)

driver = webdriver.Chrome(options=options)
url = "https://wearebraindead.com/products/brain-dead-x-homeshake-horsie-t-shirt-clay?variant=44446144331907"
url2 = 'https://wearebraindead.com/products/brain-dead-x-homeshake-horsie-lp-yellow?variant=44150010937475'
url3 = 'https://wearebraindead.com/products/oakley-factory-team-suede-flesh-lily-pad-white?variant=43320216191107'
soldout_url = 'https://wearebraindead.com/products/ms-45-x-brain-dead-t-shirt-black?variant=39705442713731'
top_list_url ='https://wearebraindead.com/collections/apparel?filter.p.m.nodnod.type=gid%3A%2F%2Fshopify%2FMetaobject%2F50877235331&pagination=5'
bottom_list_url ='https://wearebraindead.com/collections/apparel?filter.p.m.nodnod.type=gid%3A%2F%2Fshopify%2FMetaobject%2F50877268099&pagination=5'
acce_list_url = 'https://wearebraindead.com/collections/accessories?filter.p.m.nodnod.type=gid%3A%2F%2Fshopify%2FMetaobject%2F50877300867&pagination=5'
foot_list_url = 'https://wearebraindead.com/collections/footwear?pagination=5'
home_list_url = 'https://wearebraindead.com/collections/objects?pagination=5'
eye_list_url = 'https://wearebraindead.com/collections/accessories?filter.p.m.nodnod.type=gid%3A%2F%2Fshopify%2FMetaobject%2F50877333635&pagination=5'
driver.get(acce_list_url) ## 사이트 접속

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')



In [45]:

def get_price(soup):
    details = soup.find_all('section')[2]
    product_price = details.find( class_ = 'flex justify-between mb-2 text-base text-left _product__price').span.get_text()    
    return convert_currency(product_price)
    
def get_name(soup):
    h1 = soup.find('h1')
    return h1.get_text()

def get_details(soup):
  desc = soup.find('div', class_ = '_product__description').find_all('div')[1].find_all('p')
  
  description_parts = []
  
  # 색상 변수 초기화
  color = None
  material = None
  
  # 각 p 태그를 순회하면서 내용 추출
  for p in desc:
      text = p.get_text(strip=True)
      if text.startswith("Color:"):
          color = text.split("Color:")[1].strip()
      elif text.startswith("Material:"):
          material = text.split("Material:")[1].strip()
      else:
          description_parts.append(text)
  
  # description_parts를 합쳐서 하나의 문자열로 만듦
  description = "\n".join(description_parts)
  
  # 결과 딕셔너리 생성
  result = {
      "description": description,
      "color": color,
      "material" : material
  }
  return result

def get_photos(soup):
  imgs = soup.find('div', class_ = '_product__media').find_all('img')
  result = []
  for img in imgs:
    result.append(  'https:' + img['src'])
  return result

def get_sizes(soup):
  labels = soup.find('div', class_='_product__option-item').find('fieldset', class_ = '_sizes-grid').find_all('label')
  size_availability = {}
  for label in labels:
    size = label.get('aria-label').split(' ')[1]
    btn_div = label.find('div', class_='_btn')
    
    if 'open:bg-black' in btn_div.get('class'):
        size_availability[size] = False
    else:
        size_availability[size] = True
            
  return size_availability

def get_is_sold_out(soup):
  btn_text = soup.find('button', id='addToCartForm' ).find('span').get_text()
  print(btn_text)
  return btn_text == 'Notify me when available'


'Brain Dead x Homeshake Horsie LP - Yellow'

In [13]:
import json
scripts = soup.find('div', class_= '_card').find_all('a')

my_urls = []
for i, value in enumerate(scripts):
  if i % 2 == 0:
    my_url = "https://wearebraindead.com"+  value['href']
    my_urls.append(my_url)

my_urls

# json.loads(text)

['https://wearebraindead.com/collections/accessories/products/brain-dead-equipment-nylon-tennis-hat-brown?_pos=1&_fid=5a12183b3&_ss=c',
 'https://wearebraindead.com/collections/accessories/products/selvedge-denim-logohead-tote-bag-indigo-1?_pos=2&_fid=5a12183b3&_ss=c',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-x-mellow-climbing-tote-bag-natural?_pos=3&_fid=5a12183b3&_ss=c',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-x-nation-fishing-co-long-bill-mesh-hat-black?_pos=4&_fid=5a12183b3&_ss=c',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-equipment-alpine-backpack-clear-blue?_pos=5&_fid=5a12183b3&_ss=c',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-equipment-chalk-bag-black-1?_pos=6&_fid=5a12183b3&_ss=c',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-equipment-cinched-chalk-bag-blue-1?_pos=7&_fid=5a12183b3&_ss=c',
 'https://wearebraindead.com/collec

In [3]:
import json

def urls_from_json(file_path):
    # JSON 파일 열기
    with open(file_path, 'r') as file:
        urls = json.load(file)
    
    return urls
    
# JSON 파일 경로
file_path = './backup/acc_urls.json'

# 함수 호출
urls_from_json(file_path)


['https://wearebraindead.com/collections/accessories/products/selvedge-denim-logohead-tote-bag-indigo-1',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-x-mellow-climbing-tote-bag-natural',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-x-nation-fishing-co-long-bill-mesh-hat-black',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-equipment-alpine-backpack-clear-blue',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-equipment-chalk-bag-black-1',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-equipment-cinched-chalk-bag-blue-1',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-equipment-hip-bag-sprout',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-x-alterior-caterpillar-leather-belt-black',
 'https://wearebraindead.com/collections/accessories/products/brain-dead-x-alterior-leather-key-holder-black',
 'https://w